In [ ]:
import numpy as np
import pandas as pd

import statistics
import random

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

from sklearn.metrics import fbeta_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
q1features_train_1 = pd.read_pickle("q1features_train_1.pickle")
q1features_valid_1 = pd.read_pickle("q1features_valid_1.pickle")
q1target_train_1 = pd.read_pickle("q1target_train_1.pickle")
q1target_valid_1 = pd.read_pickle("q1target_valid_1.pickle")

Imputation of missing data was performed. The alternative was to use complete case analysis, which involves removing from the dataset any rows that were missing any attributes. Complete case analysis leads to a loss of information and can lead to bias if the missing data is not missing completely at random.

Single-imputation methods were used; missing categorical values were imputed using the mode of that attribute and missing numeric values were imputed using the median. The validation data set was imputed using the mode and median values of the training data set. To avoid data leakage, the training data set was not imputed using validation set data. Data leakage can lead to inaccurately high model performance (but comparatively poor performance on the test data).

In [ ]:
#Imputing validation set
for col in q1features_valid_1.columns.tolist():
  q1features_valid_1[col] = q1features_valid_1[col].replace([None], np.nan)
  if (q1features_valid_1[col].dtype == 'category' or q1features_valid_1[col].dtype =='datetime64[ns]'): #
    q1features_valid_1[col]= q1features_valid_1[col].fillna(random.choice(statistics.multimode(q1features_train_1[col])))
  if (q1features_valid_1[col].dtype == 'Int64' or q1features_valid_1[col].dtype == 'int64'):
    q1features_valid_1[col] = q1features_valid_1[col].astype('float64')
  if (q1features_valid_1[col].dtype == 'float64'):
    q1features_valid_1[col] = q1features_valid_1[col].fillna(q1features_train_1[col].median())

In [ ]:
#Imputing training set
for col in q1features_train_1.columns.tolist():
  q1features_train_1[col] = q1features_train_1[col].replace([None], np.nan)
  if (q1features_train_1[col].dtype == 'category' or q1features_train_1[col].dtype =='datetime64[ns]'): #
    q1features_train_1[col] = q1features_train_1[col].fillna(random.choice(statistics.multimode(q1features_train_1[col])))
  if (q1features_train_1[col].dtype == 'Int64' or q1features_train_1[col].dtype == 'int64'):
    q1features_train_1[col] = q1features_train_1[col].astype('float64')
  if (q1features_train_1[col].dtype == 'float64'):
    q1features_train_1[col] = q1features_train_1[col].fillna(q1features_train_1[col].median())

Numeric attributes were standardized; variable importance will be assessed using regression models, and standardization allows for easier interpretation of the model coefficients. Standardization was performed instead of normalization, as standardization is more resistant to outliers.

In [ ]:
#Standardization of validation set
for col in q1features_valid_1.columns.tolist():
  if (q1features_valid_1[col].dtype == 'Int64' or q1features_valid_1[col].dtype == 'int64' or q1features_valid_1[col].dtype == 'float64'):
    for i in range(len(q1features_valid_1[col])):
      q1features_valid_1.loc[q1features_valid_1.index[i], col] = (q1features_valid_1.loc[q1features_valid_1.index[i], col] - q1features_train_1[col].mean())/np.std(q1features_train_1[col])

In [ ]:
#Standardization  of training set
for col in q1features_train_1.columns.tolist():
  if (q1features_train_1[col].dtype == 'Int64' or q1features_train_1[col].dtype == 'int64' or q1features_train_1[col].dtype == 'float64'):
    for i in range(len(q1features_train_1[col])):
      q1features_train_1.loc[q1features_train_1.index[i], col] = (q1features_train_1.loc[q1features_train_1.index[i], col] - q1features_train_1[col].mean())/np.std(q1features_train_1[col])

In [ ]:
#The SMOTE algorithm cannot handle datetime, so converting to number of
#days since January 1st, year one
for i in range(len(q1features_train_1['INTERVIEWDATE'])):
  q1features_train_1.loc[q1features_train_1.index[i], 'INTERVIEWDATE'] =  q1features_train_1.loc[q1features_train_1.index[i], 'INTERVIEWDATE'].toordinal()

for i in range(len(q1features_valid_1['INTERVIEWDATE'])):
  q1features_valid_1.loc[q1features_valid_1.index[i], 'INTERVIEWDATE'] =  q1features_valid_1.loc[q1features_valid_1.index[i], 'INTERVIEWDATE'].toordinal()

q1features_train_1['INTERVIEWDATE'] = q1features_train_1['INTERVIEWDATE'].astype('float64')
q1features_valid_1['INTERVIEWDATE'] = q1features_valid_1['INTERVIEWDATE'].astype('float64')

In [ ]:
#Converting categorical variables to dummy variables
for col in q1features_train_1.columns.tolist():
  if (q1features_train_1[col].dtype == 'category'):
    q1features_train_1 = pd.concat([q1features_train_1, pd.get_dummies(q1features_train_1[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_train_1 = q1features_train_1.drop([col], axis=1)

In [ ]:
for col in q1features_valid_1.columns.tolist():
  if (q1features_valid_1[col].dtype == 'category'):
    q1features_valid_1 = pd.concat([q1features_valid_1, pd.get_dummies(q1features_valid_1[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_valid_1 = q1features_valid_1.drop([col], axis=1)

In [ ]:
q1target_train_1 = q1target_train_1.astype('bool')
q1target_train_1 = q1target_train_1.astype('float')

In [ ]:
q1target_valid_1 = q1target_valid_1.astype('bool')
q1target_valid_1 = q1target_valid_1.astype('float')

Multicollinearity can lead to model overfitting and can artificially hide the importance of explanatory variables. Multicollinearity (and collinearity) can be assessed using variance inflation factor values. A VIF value of over 10 indicates serious multicollinearity (Vittinghoff, E., Shiboski, S., Glidden, D., & McCulloch, C. (2004). Regression Methods in Biostatistics: Linear, Logistic, Survival and Repeated Measures Models. New York:Springer. https://doi.org/10.1007/b138825). Independent variables were iteratively removed from the model starting with the variable with the highest VIF value until all VIF values were below an accceptable threshold.

In [ ]:
#Removing highly correlated variables
vifcalcs=q1features_train_1

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = vifcalcs.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(vifcalcs.values, i)
                          for i in range(len(vifcalcs.columns))]

In [ ]:
for i in range (len(vif_data['VIF'])):
  if (vif_data.loc[vif_data.index[i], 'VIF'])> 10:
    print(vif_data.loc[vif_data.index[i], 'feature'],vif_data.loc[vif_data.index[i], 'VIF'])

In [ ]:
#unwanted = vifcalcs.columns[vifcalcs.columns.str.startswith('PRIMINSR')]
#vifcalcs.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('IDAY')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('QSTVER')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('STATE')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('_AGE65YR')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('IMONTH')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('INTERVIEWDATE')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('_AGE_G')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('_AGE80')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('HTM4')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('WTKG3')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('USENOW3')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('EDUCA')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('SLEPTIM1')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('PRIMINSR')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('FMONTH')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_1.columns[q1features_train_1.columns.str.startswith('SMOKE100')]
q1features_train_1.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('IDAY')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('QSTVER')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('STATE')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('_AGE65YR')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('IMONTH')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('INTERVIEWDATE')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('_AGE_G')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('_AGE80')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('HTM4')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('WTKG3')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('USENOW3')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('EDUCA')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('SLEPTIM1')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('PRIMINSR')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('FMONTH')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_1.columns[q1features_valid_1.columns.str.startswith('SMOKE100')]
q1features_valid_1.drop(unwanted, axis=1, inplace=True)

The dataset is imbalanced; a majority of respondents reported that they did not have COVID-19 symptoms that lasted longer than 3 months. Imbalanced data adversely affects model performance; if the model encounters few instances of the minority class then it will be unable to effectively learn from this class.

The SMOTE algorithm was used to oversample the minority class; this address the issue of imbalanced data.

In [ ]:
#SMOTE Algorithm
smo = SMOTE(random_state = 2, k_neighbors=10)
q1features_train_1_SM, q1target_train_1_SM = smo.fit_resample(q1features_train_1, q1target_train_1.ravel())

Grid search was used to optimize model hyperparameters; the models were run with each possible combination of hyperparameters across all five sliding windows. The values of these hyperparameters were taken from Dr. Aamna AlShehhi and colleague's paper 'Utilizing machine learning for survival analysis to identify risk factors for COVID-19 intensive care unit admission: A retrospective cohort study from the United Arab Emirates' (https://doi.org/10.1371/journal.pone.0291373).

The hyperparameters were selected to optimize the models' F2 scores, which is a metric that is the harmonic mean of both precision and recall, although recall is weighted heavier than precision. This metric was also used by Dr. Roman Kessler and colleagues in their study "Predictive Attributes for Developing Long COVID—A Study Using Machine Learning and Real-World Data from Primary Care Physicians in Germany" (https://doi.org/10.3390/jcm12103511). It is logical to also prioritize recall over precision in this data analytics project, as here also there are more severe consequences for false negatives than false positives.

Logistic Regression

In [ ]:
hyperparameter_score_list = []
solver_list = ['liblinear', 'newton-cg', 'newton-cholesky']

for s in solver_list:
  logr = LogisticRegression(random_state=1, solver=s)
  logReg =logr.fit(q1features_train_1_SM, q1target_train_1_SM)
  logRegPrediction = logReg.predict(q1features_valid_1)
  logRegPrediction = np.where(logRegPrediction > 0.5, 1, 0)
  f2_score = fbeta_score(q1target_valid_1, logRegPrediction, beta=2)
  hyperparameter_score_list.append((s, f2_score))

logRegScores1 = pd.DataFrame(hyperparameter_score_list, columns =['Solver', 'F2_Score'])

Naive Bayes

In [ ]:
# Build a Gaussian Classifier
model = GaussianNB()

In [ ]:
# Model training
nbModel = model.fit(q1features_train_1_SM, q1target_train_1_SM)

In [ ]:
#Making predictions on the validation set
nbPrediction = nbModel.predict(q1features_valid_1)

In [ ]:
f2_scoreNB1 = fbeta_score(q1target_valid_1, nbPrediction, beta=2)

Random Forest

In [ ]:
hyperparameter_score_list = []
n_estimators_list =[25, 50, 75, 100]
max_depth_list = [1, 2, 3, 4, 5, 10]
criterion_list = ['gini', 'entropy', 'log_loss']

for n in n_estimators_list:
  for m in max_depth_list:
    for c in criterion_list:
      rf = RandomForestClassifier(n_estimators=n, max_depth=m, criterion=c, random_state=42)
      rf.fit(q1features_train_1_SM, q1target_train_1_SM)
      rfPrediction = rf.predict(q1features_valid_1)
      f2_score = fbeta_score(q1target_valid_1, rfPrediction, beta=2)
      hyperparameter_score_list.append((n, m, c, f2_score))

rfScores1 = pd.DataFrame(hyperparameter_score_list, columns =['n_Estimators', 'Max_depth', 'Criterion', 'F2_Score'])

NameError: name 'RandomForestClassifier' is not defined

Gradient-Boosted Trees

In [ ]:
hyperparameter_score_list = []
n_estimators_list =[25, 50, 75, 100]
max_depth_list = [1, 2, 3, 4, 5, 10]
subsample_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]

for n in n_estimators_list:
  for m in max_depth_list:
      for s in subsample_list:
        bst = GradientBoostingClassifier(n_estimators=n, max_depth=m, subsample=s, random_state=42).fit(q1features_train_1_SM, q1target_train_1_SM)
        bstPrediction = bst.predict(q1features_valid_1)
        f2_score = fbeta_score(q1target_valid_1, bstPrediction, beta=2)
        hyperparameter_score_list.append((n, m, s, f2_score))

bstScores1 = pd.DataFrame(hyperparameter_score_list, columns =['n_Estimators', 'Max_depth', 'Subsample', 'F2_Score'])

In [ ]:
logRegScores1.to_pickle('logRegScores1.pickle')
f2_scoreNB1.to_pickle('f2_scoreNB1.pickle')
rfScores1.to_pickle('rfScores1.pickle')
bstScores1.to_pickle('bstScores1.pickle')